In [ ]:
from agent_framework import ChatAgent
from agent_framework.openai import OpenAIChatClient
from agent_framework.devui import serve

def get_weather(location: str) -> str:
    """Get weather for a location."""
    return f"Weather in {location}: 72°F and sunny"

# Create your agent
agent = ChatAgent(
    name="WeatherAgent",
    chat_client=OpenAIChatClient(),
    tools=[get_weather]
)

# Launch debug UI - that's it!
serve(entities=[agent], auto_open=True)
# → Opens browser to http://localhost:8080

## Demo: customizing OpenAI runs
This cell shows how to wire up `OpenAIPromptExecutionSettings` so you can fine-tune temperature, token budgets, penalties, and even force JSON output when calling Semantic Kernel.

In [ ]:
import asyncio
import os

from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import (
    AzureOpenAIChatCompletion,
    OpenAIPromptExecutionSettings,
)


async def run_prompt_with_settings() -> None:
    """Invoke an Azure OpenAI chat deployment with explicit execution settings."""
    kernel = Kernel()

    chat_service = AzureOpenAIChatCompletion(
        service_id="gpt5-demo",
        api_key=os.environ["AZURE_OPENAI_API_KEY"],
        endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
        deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT", "gpt-4o-mini"),
        api_version=os.getenv("AZURE_OPENAI_API_VERSION", "2024-06-01"),
    )
    kernel.add_service(chat_service)

    execution_settings = OpenAIPromptExecutionSettings(
        temperature=0.2,
        top_p=0.9,
        presence_penalty=0.0,
        frequency_penalty=0.0,
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "sk_summary",
                "schema": {
                    "type": "object",
                    "properties": {
                        "scenario": {"type": "string"},
                        "impact": {"type": "string"},
                        "confidence": {
                            "type": "number",
                            "minimum": 0.0,
                            "maximum": 1.0,
                        },
                    },
                    "required": ["scenario", "impact"],
                    "additionalProperties": False,
                },
            },
        },
        max_input_tokens=6000,
        max_output_tokens=256,
    )

    prompt = (
        "Summarize one realistic Semantic Kernel use case and report its expected impact.\n"
        "Return the JSON payload requested by the schema."
    )

    response = await kernel.invoke_prompt(
        prompt=prompt,
        service_id=chat_service.service_id,
        execution_settings=execution_settings,
    )

    print(response)


asyncio.run(run_prompt_with_settings())